# Ember Model Registry Tutorial

This notebook provides a comprehensive guide to using Ember's Model Registry system. You'll learn how to:

1. Initialize the model registry
2. Configure and use different LLM providers
3. Access models through different API patterns
4. Track usage and manage costs
5. Create custom providers
6. Handle advanced scenarios like streaming and concurrency

Let's get started!

## 1. Setup and Initialization

First, let's import the necessary components and initialize the model registry.

In [ ]:
import os
import logging
from typing import Dict, List, Optional

# Configure logging
logging.basicConfig(level=logging.INFO)

# Import Ember components
from ember.core.registry.model.initialization import initialize_registry
from ember.core.registry.model.base.services.model_service import ModelService, create_model_service
from ember.core.registry.model.base.services.usage_service import UsageService
from ember.core.registry.model.config.model_enum import ModelEnum
from ember.api import models  # High-level API

### 1.1 Environment Setup

To use LLM providers, you need to set up your API keys. In a production environment, these would be set as environment variables or in a configuration file. For this notebook, we'll set them directly.

In [ ]:
# Set up API keys (replace with your own or use environment variables)
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your key
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"  # Replace with your key
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"  # Replace with your key

# For safety in the notebook, we'll check if keys are actually set
providers_available = {
    "OpenAI": bool(os.environ.get("OPENAI_API_KEY")),
    "Anthropic": bool(os.environ.get("ANTHROPIC_API_KEY")),
    "Google/Deepmind": bool(os.environ.get("GOOGLE_API_KEY"))
}

print("Available providers:")
for provider, available in providers_available.items():
    print(f"- {provider}: {'✅' if available else '❌'}")

### 1.2 Initialize the Model Registry

Now we'll initialize the model registry, which will discover available models from configured providers.

In [ ]:
# Initialize the registry with auto-discovery enabled
registry = initialize_registry(auto_discover=True)

# Create a model service with usage tracking
usage_service = UsageService()
model_service = create_model_service(
    registry=registry,
    usage_service=usage_service
)

# List available models
available_models = registry.list_models()
print(f"Discovered {len(available_models)} models:")
for model in available_models[:10]:  # Show first 10 models
    print(f"- {model}")
if len(available_models) > 10:
    print(f"... and {len(available_models) - 10} more")

## 2. Basic Usage Patterns

Ember provides multiple ways to interact with models. Let's explore the different access patterns.

### 2.1 Direct Model Service Usage

The most basic approach is to use the model service directly.

In [ ]:
# Select a model that's available based on our environment check
model_id = None
if providers_available["OpenAI"]:
    model_id = "openai:gpt-4o"
elif providers_available["Anthropic"]:
    model_id = "anthropic:claude-3-5-sonnet"
elif providers_available["Google/Deepmind"]:
    model_id = "deepmind:gemini-1.5-pro"
else:
    print("No providers available. Using mock model for demonstration.")
    model_id = "mock:text-model"

# Use the model service to invoke the model
if model_id:
    response = model_service.invoke_model(
        model_id=model_id,
        prompt="What is the Ember framework?",
        temperature=0.7,
        max_tokens=150
    )
    
    print(f"Response from {model_id}:\n{response.data}\n")
    print(f"Usage: {response.usage.total_tokens} tokens")

### 2.2 Function-Style API

For a more concise approach, you can use the function-style API.

In [ ]:
# Use the high-level API
try:
    if providers_available["OpenAI"]:
        response = models.openai.gpt4o("What are the benefits of using Networks of Networks (NONs)?")
        print(f"OpenAI GPT-4o response:\n{response.data}\n")
    
    if providers_available["Anthropic"]:
        response = models.anthropic.claude_3_5_sonnet("How does Ember help with LLM orchestration?")
        print(f"Anthropic Claude response:\n{response.data}\n")
except Exception as e:
    print(f"Error using function-style API: {e}")

### 2.3 Enum-Based Access

For type safety and IDE support, you can use enum-based access.

In [ ]:
# Use enum-based access
try:
    response = model_service(ModelEnum.GPT4O, "What are model enumerations good for?")
    print(f"Response using enum access:\n{response.data}\n")
except Exception as e:
    print(f"Error using enum-based access: {e}")

## 3. Provider Integration

Let's explore how Ember integrates with different providers and how you can configure provider-specific parameters.

### 3.1 Provider-Specific Parameters

Each provider has specific parameters you can configure.

In [ ]:
# OpenAI-specific parameters
if providers_available["OpenAI"]:
    response = model_service.invoke_model(
        model_id="openai:gpt-4o",
        prompt="Generate a creative story idea.",
        temperature=0.9,  # Higher creativity
        max_tokens=200,
        provider_params={
            "top_p": 0.95,
            "presence_penalty": 0.5,
            "frequency_penalty": 0.5
        }
    )
    print(f"OpenAI response with custom parameters:\n{response.data}\n")

# Anthropic-specific parameters
if providers_available["Anthropic"]:
    response = model_service.invoke_model(
        model_id="anthropic:claude-3-5-sonnet",
        prompt="Generate a creative story idea.",
        temperature=0.9,
        max_tokens=200,
        provider_params={
            "top_k": 40,
            "top_p": 0.95,
            "stop_sequences": ["THE END"]
        }
    )
    print(f"Anthropic response with custom parameters:\n{response.data}\n")

### 3.2 Model Information and Metadata

You can access detailed information about registered models.

In [ ]:
# Get model info for a specific model
if model_id:
    model_info = registry.get_model_info(model_id)
    if model_info:
        print(f"Model ID: {model_info.id}")
        print(f"Model Name: {model_info.name}")
        print(f"Provider: {model_info.provider.name}")
        print(f"Input Cost: ${model_info.cost.input_cost_per_thousand/1000:.6f} per token")
        print(f"Output Cost: ${model_info.cost.output_cost_per_thousand/1000:.6f} per token")
        
        if model_info.provider.custom_args:
            print("\nProvider Custom Args:")
            for key, value in model_info.provider.custom_args.items():
                print(f"- {key}: {value}")
    else:
        print(f"No model info found for {model_id}")

## 4. Model Discovery and Registration

Let's explore how the model discovery process works and how to manually register models.

### 4.1 Manual Model Registration

You can manually register models with the registry.

In [ ]:
from ember.core.registry.model.base.schemas.model_info import ModelInfo, ProviderInfo
from ember.core.registry.model.base.schemas.cost import ModelCost, RateLimit

# Create a custom model info
custom_model_info = ModelInfo(
    id="custom:my-model",
    name="Custom Model",
    provider=ProviderInfo(
        name="Custom",
        default_api_key="mock-api-key"
    ),
    cost=ModelCost(
        input_cost_per_thousand=0.001,
        output_cost_per_thousand=0.002
    ),
    rate_limit=RateLimit(
        tokens_per_minute=100000,
        requests_per_minute=3000
    )
)

# Register the custom model
try:
    registry.register_model(custom_model_info)
    print(f"Successfully registered custom model: {custom_model_info.id}")
    
    # Verify it's in the list of models
    if custom_model_info.id in registry.list_models():
        print("Custom model is now in the registry")
except Exception as e:
    print(f"Error registering custom model: {e}")

### 4.2 Model Discovery Process

Let's trigger a model discovery to see how it works.

In [ ]:
# Trigger model discovery
print("Initiating model discovery...")
new_models = registry.discover_models()

if new_models:
    print(f"Discovered {len(new_models)} new models:")
    for model in new_models:
        print(f"- {model}")
else:
    print("No new models discovered (all models were already registered)")

## 5. Usage Tracking

One of the powerful features of Ember's model registry is built-in usage tracking.

In [ ]:
# Make some model calls to generate usage data
if model_id:
    prompts = [
        "What is artificial intelligence?",
        "Explain quantum computing in simple terms.",
        "What are the ethical implications of AI?"
    ]
    
    for i, prompt in enumerate(prompts, 1):
        try:
            print(f"Generating response {i}/3...")
            model_service.invoke_model(
                model_id=model_id,
                prompt=prompt,
                max_tokens=100  # Keep responses short for demo
            )
        except Exception as e:
            print(f"Error invoking model: {e}")
    
    # Get usage statistics
    total_usage = usage_service.get_total_usage()
    print("\nUsage Statistics:")
    print(f"Total Tokens: {total_usage.total_tokens}")
    print(f"Prompt Tokens: {total_usage.prompt_tokens}")
    print(f"Completion Tokens: {total_usage.completion_tokens}")
    print(f"Estimated Cost: ${total_usage.cost:.6f}")
    
    # Get usage by model
    usage_by_model = usage_service.get_usage_by_model()
    print("\nUsage by Model:")
    for model_id, usage in usage_by_model.items():
        print(f"- {model_id}: {usage.total_tokens} tokens, ${usage.cost:.6f}")

## 6. Custom Provider

Advanced users can create custom providers to integrate with other LLM services.

In [ ]:
from ember.core.registry.model.providers.base_provider import BaseProviderModel, BaseChatParameters
from ember.core.registry.model.base.schemas.chat_schemas import ChatRequest, ChatResponse
from ember.core.registry.model.base.schemas.usage import UsageStats

# Define a simple custom provider
class CustomProviderModel(BaseProviderModel):
    # Set the provider name (required for discovery and registration)
    PROVIDER_NAME = "Custom"
    
    def create_client(self) -> Any:
        # In a real provider, you would initialize your API client here
        # For this example, we'll just return a dummy client
        return {"api_key": self.model_info.provider.default_api_key}
    
    def forward(self, request: ChatRequest) -> ChatResponse:
        # In a real provider, you would call your API here
        # For this example, we'll just echo the prompt
        response_text = f"[CUSTOM PROVIDER] You asked: {request.prompt}\n\nThis is a simulated response from {self.model_info.name}."
        
        # Calculate basic usage stats
        prompt_tokens = len(request.prompt.split())
        completion_tokens = len(response_text.split())
        
        usage = UsageStats(
            prompt_tokens=prompt_tokens,
            completion_tokens=completion_tokens,
            total_tokens=prompt_tokens + completion_tokens,
            cost_usd=0.0  # Simulated cost
        )
        
        return ChatResponse(
            data=response_text,
            raw_output={"simulated": True},
            usage=usage
        )

# Register the custom provider with the model factory
from ember.core.registry.model.base.registry.factory import ModelFactory

try:
    ModelFactory.register_custom_provider(
        provider_name="Custom",
        provider_class=CustomProviderModel
    )
    print("Successfully registered custom provider")
    
    # Now we can use our custom provider
    response = model_service.invoke_model(
        model_id="custom:my-model",
        prompt="This is a test of my custom provider"
    )
    
    print(f"\nResponse from custom provider:\n{response.data}")
except Exception as e:
    print(f"Error with custom provider: {e}")

## 7. Advanced Topics

Let's explore some more advanced topics like error handling and concurrency.

### 7.1 Error Handling

Proper error handling is essential for robust applications.

In [ ]:
# Attempt to invoke a non-existent model
try:
    response = model_service.invoke_model(
        model_id="nonexistent:model",
        prompt="This should fail"
    )
except Exception as e:
    print(f"Expected error: {e}")
    print("This is good! The error was caught properly.")

# Error handling with invalid parameters
try:
    response = model_service.invoke_model(
        model_id=model_id,
        prompt="This should fail",
        temperature=3.0  # Invalid temperature (should be between 0 and 2)
    )
except Exception as e:
    print(f"\nExpected error with invalid parameters: {e}")

### 7.2 Concurrency

Ember's model registry is designed to be thread-safe for concurrent operations.

In [ ]:
import concurrent.futures
import time

def invoke_model(prompt):
    try:
        start_time = time.time()
        response = model_service.invoke_model(
            model_id=model_id,
            prompt=prompt,
            max_tokens=50  # Keep responses short for demo
        )
        duration = time.time() - start_time
        return {
            "prompt": prompt,
            "response": response.data[:50] + "...",  # Truncate for display
            "duration": duration,
            "success": True
        }
    except Exception as e:
        return {
            "prompt": prompt,
            "error": str(e),
            "success": False
        }

# Set up concurrent requests
prompts = [
    "What is the capital of France?",
    "Who wrote Romeo and Juliet?",
    "What is the formula for water?",
    "What is the tallest mountain in the world?"
]

if model_id:
    print("Executing concurrent model invocations...")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(invoke_model, prompt) for prompt in prompts]
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
    print("\nConcurrent invocation results:")
    for i, result in enumerate(results, 1):
        if result["success"]:
            print(f"Request {i}: {result['prompt']}")
            print(f"Response: {result['response']}")
            print(f"Duration: {result['duration']:.2f} seconds\n")
        else:
            print(f"Request {i}: {result['prompt']}")
            print(f"Error: {result['error']}\n")

## 8. Best Practices

Let's wrap up by discussing some best practices for using the model registry.

### Best Practices for Using Ember Model Registry

1. **API Key Management**
   - Store API keys in environment variables or secure configuration
   - Rotate keys regularly according to provider recommendations
   - Use different keys for development, testing, and production

2. **Cost Management**
   - Leverage usage tracking to monitor costs
   - Set token limits appropriate for your use case
   - Use lower-cost models for tasks that don't require high capability

3. **Error Handling**
   - Always implement proper error handling
   - Have fallback strategies for API outages
   - Implement retry logic with exponential backoff

4. **Performance Optimization**
   - Use concurrent requests for independent operations
   - Implement caching for common queries
   - Set appropriate timeouts to prevent hanging requests

5. **Testing**
   - Create mock providers for testing
   - Use deterministic responses in test environments
   - Set up test fixtures with known inputs and outputs

## 9. Conclusion

In this notebook, we've explored the Ember Model Registry's capabilities:

1. We learned how to initialize and configure the model registry
2. We explored different API patterns for model invocation
3. We saw how to work with provider-specific parameters
4. We demonstrated model discovery and manual registration
5. We tracked and visualized usage statistics
6. We created a custom provider implementation
7. We handled errors and concurrent operations

The Ember Model Registry provides a flexible and powerful foundation for building LLM applications with multiple providers. It simplifies provider integration, standardizes model access, and provides essential features like usage tracking and error handling.